# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
part1_file = "weather_df.csv"

types_df = pd.read_csv(part1_file)

vacation_data_df = types_df.drop(columns=["Unnamed: 0"])
vacation_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,mataura,100,NZ,1618277964,95,-46.1927,168.8643,55.00,3.00
1,albany,100,US,1618277821,75,42.6001,-73.9662,50.00,4.52
2,anar darreh,0,FI,1618277983,93,68.9060,27.0288,17.60,1.14
3,jamestown,90,US,1618277856,81,42.0970,-79.2353,50.00,6.91
4,ginir,100,ET,1618277983,71,7.1333,40.7000,58.46,5.61
...,...,...,...,...,...,...,...,...,...
547,chattanooga,75,US,1618278177,33,35.0456,-85.3097,73.40,2.86
548,makokou,92,GA,1618278251,96,0.5738,12.8642,70.20,1.10
549,champerico,8,GT,1618278252,94,14.3000,-91.9167,69.01,11.36
550,mukhen,100,RU,1618278253,97,48.1000,136.1000,35.80,7.67


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = vacation_data_df[["Lat", "Lng"]]

humidity = vacation_data_df["Humidity"].astype(float)

fig = gmaps.figure(center = [0,0], zoom_level = 2)

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                                 dissipating=False, max_intensity=100, 
                                 point_radius=1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [11]:
new_vacation_df = vacation_data_df.loc[(vacation_data_df["Wind Speed"] <= 10) & (vacation_data_df["Cloudiness"] == 0) & \
                                   (vacation_data_df["Max Temp"] >= 70) & (vacation_data_df["Max Temp"] <= 80)].dropna()

new_vacation_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
214,doha,0,QA,1618278088,69,25.2867,51.5333,78.80,3.44
292,nizwa,0,OM,1618278132,36,22.9333,57.5333,73.74,1.03
337,manicore,0,BR,1618278152,94,-5.8092,-61.3003,78.80,1.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [16]:
hotel_df = pd.DataFrame(new_vacation_df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
214,doha,QA,25.2867,51.5333,
292,nizwa,OM,22.9333,57.5333,
337,manicore,BR,-5.8092,-61.3003,


In [17]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        
        print("MISSING RESULT. SKIPPING...")

Retrieving Results for Index 214: doha.
Closest hotel in doha is The Curve Hotel.
Retrieving Results for Index 292: nizwa.
Closest hotel in nizwa is Antique Inn - Nizwa.
Retrieving Results for Index 337: manicore.
Closest hotel in manicore is Hotel Durval.


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))